In [1]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import sys
sys.path.append("../../lib")
from metrics import levenshtein
import pickle

In [2]:
folder = "../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_training_18M_without_Finnish/ES/"

In [3]:
output_folder = Path("../../data/es")

In [4]:
files = sorted(os.listdir(folder))
len(files)

1

In [5]:
import glob

files = glob.glob(folder + '/**/*.txt', recursive=True)
len(files)

147

In [6]:
from multiprocessing import Pool

def extract(name):
    with open(name) as file:
        return file.readlines()
    
def create_windows(x):
    A, B, window_length = x
    assert len(A) == len(B)
    return [(A[i:i + window_length], B[i:i + window_length]) 
            for i in range(len(A) + 1)]
    
p = Pool(4)
    
data = list(p.imap_unordered(extract, tqdm(files), chunksize = 128))
len(data)

  0%|          | 0/147 [00:00<?, ?it/s]

147

In [7]:
# data = []
# for f in tqdm(files):
#     with open(f) as file:
#         data.append(file.readlines())

data = pd.DataFrame(data, 
                    columns = ["ocr_to_input", 
                               "ocr_aligned", 
                               "gs_aligned"])\
.assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
        ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
        gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

print(data.shape)
data.head()

(147, 3)


,ocr_to_input,ocr_aligned,gs_aligned
0,"A MI dón. Vease. Es voz andquada,y la trabe Ne...","@@@@@A MI@ dón. Vease. Es voz andquada,y la tr...","AMI A MIGAJAS. Vease@ @@Migája. AMIGO,@@@@ @@..."
1,& V S A Í1T7 iCnmulo <t U Señora 'Dona diaria ...,@@@@@@@@@@@@@@@@@@@@@@@@& @@@@@V @@S @@@A Í1T7...,© Biblioteca Nacional de España Tu- MVSA III....
2,De Don Pedro Calderón déla Barca.' No soy Alva...,@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@De Don Pedro ...,© Biblioteca Nacional de España De Don Pedro ...
3,EL REY. Or quanto por parte de vos la Real Aca...,@EL REY. Or quanto por parte de vos la Real Ac...,EL REY. Or quanto por parte de vos la Real Ac...
4,© Biblioteca Nacional de España w O i i i Come...,© Bibli@oteca @@@Na@@cio@@nal @@@@@@de España ...,"@ Alarico. Vn soldado Romano, pregundo por ti ..."


In [8]:
data.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,147.000000,147.000000,147.000000
mean,2937.489796,3247.673469,3247.673469
std,1925.402795,2212.369396,2212.369396
min,261.000000,262.000000,262.000000
25%,1277.500000,1433.500000,1433.500000
50%,1861.000000,1999.000000,1999.000000
75%,5260.500000,5780.000000,5780.000000
max,5953.000000,6993.000000,6993.000000


In [9]:
levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
            hypothesis = data.ocr_to_input).cer.describe()

count     147.000000
mean       76.538615
std       450.688649
min         4.203205
25%        11.185187
50%        27.146042
75%        55.906375
max      5452.173913
Name: cer, dtype: float64

In [10]:
levenshtein(reference = data.gs_aligned, 
            hypothesis = data.ocr_aligned).cer.describe()

count    147.000000
mean      30.835365
std       21.797847
min        4.166667
25%       10.892862
50%       25.373394
75%       46.905675
max       98.275862
Name: cer, dtype: float64

In [11]:
vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))
with open(output_folder/"data/vocabulary.pkl", "wb") as file:
    pickle.dump(vocabulary, file)

179


In [12]:
dev = data.sample(n = 5, random_state = 1)
dev.to_pickle(output_folder/"data/dev.pkl")
dev.shape

(5, 3)

In [13]:
train = data.drop(dev.index)
train.to_pickle(output_folder/"data/train.pkl")
train.shape

(142, 3)

In [23]:
train.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,142.000000,142.000000,142.000000
mean,2966.626761,3282.007042,3282.007042
std,1918.321080,2207.777575,2207.777575
min,311.000000,313.000000,313.000000
25%,1281.250000,1457.750000,1457.750000
50%,1876.000000,2033.000000,2033.000000
75%,5261.750000,5786.000000,5786.000000
max,5953.000000,6993.000000,6993.000000


In [14]:
dev.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,5.000000,5.000000,5.000000
mean,2110.000000,2272.600000,2272.600000
std,2169.658498,2367.143384,2367.143384
min,261.000000,262.000000,262.000000
25%,1208.000000,1255.000000,1255.000000
50%,1437.000000,1498.000000,1498.000000
75%,1787.000000,1994.000000,1994.000000
max,5857.000000,6354.000000,6354.000000


In [15]:
levenshtein(reference = dev.gs_aligned.str.replace("@", ""), 
            hypothesis = dev.ocr_to_input).cer.describe()

count      5.000000
mean     167.521258
std      296.806703
min       10.664479
25%       16.410615
50%       49.858677
75%       63.702822
max      696.969697
Name: cer, dtype: float64

In [16]:
levenshtein(reference = dev.gs_aligned, 
            hypothesis = dev.ocr_to_input).cer.describe()

count     5.000000
mean     43.407167
std      31.910011
min      10.358566
25%      15.687583
50%      44.232698
75%      58.970727
max      87.786260
Name: cer, dtype: float64

In [17]:
window_length = 100

In [18]:
df = train#.head(100)
train_aligned = list(p.imap_unordered(create_windows, 
                                      tqdm(zip(df.ocr_aligned, 
                                               df.gs_aligned, 
                                               [window_length for x in df.ocr_aligned]), 
                                           total = len(df.ocr_aligned)),
                                      chunksize = 128))
s = []
for r in tqdm(train_aligned):
    s.extend(r)
train_aligned = pd.DataFrame(s, columns = ["source", "target"])
print(train_aligned.shape)
train_aligned.head()

  0%|          | 0/142 [00:00<?, ?it/s]

  0%|          | 0/142 [00:00<?, ?it/s]

(466187, 2)


,source,target
0,"LIBRO V. nian en la corte, assí para grande za...",@@@ CAP. @XI. LIBRO @@@@V. @@DE @LOS @@@@llos....
1,"IBRO V. nian en la corte, assí para grande za ...",@@ CAP. @XI. LIBRO @@@@V. @@DE @LOS @@@@llos. ...
2,"BRO V. nian en la corte, assí para grande za d...",@ CAP. @XI. LIBRO @@@@V. @@DE @LOS @@@@llos. D...
3,"RO V. nian en la corte, assí para grande za de...",CAP. @XI. LIBRO @@@@V. @@DE @LOS @@@@llos. De...
4,"O V. nian en la corte, assí para grande za del...",CAP. @XI. LIBRO @@@@V. @@DE @LOS @@@@llos. De ...


In [19]:
train_aligned = train_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
train_aligned.head()

,source,target
0,"LIBRO V. nian en la corte, assí para grande za...",@@@ CAP. @XI. LIBRO @@@@V. @@DE @LOS @@@@llos....
1,"IBRO V. nian en la corte, assí para grande za ...",@@ CAP. @XI. LIBRO @@@@V. @@DE @LOS @@@@llos. ...
2,"BRO V. nian en la corte, assí para grande za d...",@ CAP. @XI. LIBRO @@@@V. @@DE @LOS @@@@llos. D...
3,"RO V. nian en la corte, assí para grande za de...",CAP. @XI. LIBRO @@@@V. @@DE @LOS @@@@llos. De...
4,"O V. nian en la corte, assí para grande za del...",CAP. @XI. LIBRO @@@@V. @@DE @LOS @@@@llos. De ...


In [20]:
dev_aligned = dev.apply(lambda r: create_windows((r["ocr_aligned"], r["gs_aligned"], window_length)), 
                            axis = 1).sum()
dev_aligned = pd.DataFrame(dev_aligned, columns = ["source", "target"])
print(dev_aligned.shape)
dev_aligned.head()

(11368, 2)


,source,target
0,"m v ak* vi. Hamc faltado modorra, En iendo el ...","@ @MVSA VI. Hame faltado modorra, En iendo el ..."
1,"v ak* vi. Hamc faltado modorra, En iendo el r...","@MVSA VI. Hame faltado modorra, En iendo el r..."
2,"v ak* vi. Hamc faltado modorra, En iendo el re...","@MVSA VI. Hame faltado modorra, En iendo el re..."
3,"ak* vi. Hamc faltado modorra, En iendo el ret...","MVSA VI. Hame faltado modorra, En iendo el ret..."
4,"ak* vi. Hamc faltado modorra, En iendo el reto...","VSA VI. Hame faltado modorra, En iendo el reto..."


In [21]:
dev_aligned = dev_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
dev_aligned.head()

,source,target
0,"m v ak* vi. Hamc faltado modorra, En iendo el ...","@ @MVSA VI. Hame faltado modorra, En iendo el ..."
1,"v ak* vi. Hamc faltado modorra, En iendo el r...","@MVSA VI. Hame faltado modorra, En iendo el r..."
2,"v ak* vi. Hamc faltado modorra, En iendo el re...","@MVSA VI. Hame faltado modorra, En iendo el re..."
3,"ak* vi. Hamc faltado modorra, En iendo el ret...","MVSA VI. Hame faltado modorra, En iendo el ret..."
4,"ak* vi. Hamc faltado modorra, En iendo el reto...","VSA VI. Hame faltado modorra, En iendo el reto..."


In [22]:
train_aligned.to_pickle(output_folder/"data/train_aligned.pkl")
dev_aligned.to_pickle(output_folder/"data/dev_aligned.pkl")